In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [7]:
train = pd.read_csv("train.csv")
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [9]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : "Missing Values", 1 : "% of Total Values"})
    return mis_val_table_ren_columns
missing_values_table(train)

,Missing Values,% of Total Values
Id,0,0.000000
MSSubClass,0,0.000000
MSZoning,0,0.000000
LotFrontage,259,17.739726
LotArea,0,0.000000
...,...,...
MoSold,0,0.000000
YrSold,0,0.000000
SaleType,0,0.000000
SaleCondition,0,0.000000


In [65]:
df_null = train.isnull().sum() * 100 / len(train)
df_null_cols = df_null[df_null > 20].index
train1 = train.drop(df_null_cols, axis=1)
train1.shape


(1460, 76)

In [11]:
# df.dropna(thresh=df.shape[0]*0.6,how='all',axis=1)
# df.drop(df.columns[df.apply(lambda col: (col.isnull().sum()/len(df)) > 0.20)], axis=1, inplace=True)

In [12]:
# sns.pairplot(train1)
# cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
# sns.pairplot(df_train[cols])
# plt.show();

In [78]:
'''def nans(df): 
    return df[df.isnull().any(axis=1)]

train3 = train1.drop(nans(train1).index, axis=0)
train3.shape'''



(1094, 76)

In [43]:
train2 = train1.dropna()
train2.shape

(1094, 76)

In [33]:
train2["HouseStyle"].value_counts()

1Story    540
2Story    346
1.5Fin    117
SLvl       43
SFoyer     23
1.5Unf     10
2.5Unf      9
2.5Fin      6
Name: HouseStyle, dtype: int64

In [36]:
train2[["SalePrice", "LotArea", 'YearRemodAdd', 'YearBuilt',
'OverallQual', 'OverallCond', 'GarageArea', 'OpenPorchSF', 'EnclosedPorch',
'PoolArea']].corr()

,SalePrice,LotArea,YearRemodAdd,YearBuilt,OverallQual,OverallCond,GarageArea,OpenPorchSF,EnclosedPorch,PoolArea
SalePrice,1.000000,0.302268,0.519806,0.523434,0.795437,-0.138511,0.620772,0.338600,-0.161711,0.092085
LotArea,0.302268,1.000000,0.024308,0.028954,0.169876,-0.033113,0.213104,0.098051,-0.022789,0.109294
YearRemodAdd,0.519806,0.024308,1.000000,0.625905,0.568582,0.024427,0.413917,0.259469,-0.220843,0.018813
YearBuilt,0.523434,0.028954,0.625905,1.000000,0.590761,-0.437647,0.470809,0.235486,-0.399540,0.006402
OverallQual,0.795437,0.169876,0.568582,0.590761,1.000000,-0.189587,0.555315,0.335884,-0.155080,0.080037
OverallCond,-0.138511,-0.033113,0.024427,-0.437647,-0.189587,1.000000,-0.233585,-0.084405,0.067124,-0.024919
GarageArea,0.620772,0.213104,0.413917,0.470809,0.555315,-0.233585,1.000000,0.303611,-0.117481,0.081126
OpenPorchSF,0.338600,0.098051,0.259469,0.235486,0.335884,-0.084405,0.303611,1.000000,-0.132970,0.033044
EnclosedPorch,-0.161711,-0.022789,-0.220843,-0.399540,-0.155080,0.067124,-0.117481,-0.132970,1.000000,0.076672
PoolArea,0.092085,0.109294,0.018813,0.006402,0.080037,-0.024919,0.081126,0.033044,0.076672,1.000000


In [37]:
Y = train2["SalePrice"]
X = train2[["LotArea", 'YearRemodAdd', 'YearBuilt','OverallQual', 'OverallCond', 'GarageArea', 'OpenPorchSF', 'EnclosedPorch',
'PoolArea']]
X = sm.add_constant(X)
model = sm.OLS(Y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     290.5
Date:                Thu, 12 Nov 2020   Prob (F-statistic):          1.26e-281
Time:                        21:23:58   Log-Likelihood:                -13274.
No. Observations:                1094   AIC:                         2.657e+04
Df Residuals:                    1084   BIC:                         2.662e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -7.564e+05    1.7e+05     -4.460      0.000   -1.09e+06   -4.24e+05
LotArea           1.5185      0.173      8.758      0.000       1.178       1.859
YearRemodAdd    183.4666     98.416      1.864      0.063      -9.640     376.573
YearBuilt       135.5323     78.530      1.726      0.085     -18.555     289.620
OverallQual    3.572e+04   1436.340     24.866      0.000    3.29e+04    3.85e+04
OverallCond    4223.3521   1608.423      2.626      0.009    1067.378    7379.326
GarageArea       94.4955      9.048     10.444      0.000      76.743     112.248
OpenPorchSF      50.4370     22.807      2.211      0.027       5.685      95.189
EnclosedPorch   -13.9376     24.891     -0.560      0.576     -62.777      34.902
PoolArea         21.7372     34.027      0.639      0.523     -45.030      88.504
==============================================================================
Omnibus:                      478.399   Durbin-Watson:                   1.897
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12373.582
Skew:                           1.435   Prob(JB):                         0.00
Kurtosis:                      19.224   Cond. No.                     1.64e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.64e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
Y = train2["SalePrice"]
X = train2[["LotArea", 'YearRemodAdd', 'YearBuilt','OverallQual', 'OverallCond', 'GarageArea', 'OpenPorchSF']]
X = sm.add_constant(X)
model = sm.OLS(Y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     373.9
Date:                Thu, 12 Nov 2020   Prob (F-statistic):          4.70e-284
Time:                        21:27:16   Log-Likelihood:                -13274.
No. Observations:                1094   AIC:                         2.656e+04
Df Residuals:                    1086   BIC:                         2.660e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -7.785e+05   1.63e+05     -4.774      0.000    -1.1e+06   -4.59e+05
LotArea          1.5318      0.172      8.881      0.000       1.193       1.870
YearRemodAdd   179.5438     98.085      1.830      0.067     -12.914     372.001
YearBuilt      150.2233     72.401      2.075      0.038       8.161     292.286
OverallQual   3.569e+04   1427.202     25.007      0.000    3.29e+04    3.85e+04
OverallCond   4340.0984   1587.013      2.735      0.006    1226.139    7454.058
GarageArea      94.4716      9.026     10.467      0.000      76.762     112.181
OpenPorchSF     51.5176     22.710      2.269      0.023       6.958      96.077
==============================================================================
Omnibus:                      494.981   Durbin-Watson:                   1.893
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12656.373
Skew:                           1.512   Prob(JB):                         0.00
Kurtosis:                      19.386   Cond. No.                     1.58e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""